In [181]:
from ftapi import *
from datetime import datetime
from IPython.display import clear_output

import pandas as pd

api = Api(u,s)

token generated. Expires in: 7200 seconds


In [2]:
CURSUS_42_INNER_CIRCLE = {
     '42cursus-libft': 1314,
     'minirt': 1315,
     '42cursus-ft_printf': 1316,
     'netwhat': 1318,
     'exam-rank-02': 1320,
     'exam-rank-03': 1321,
     'exam-rank-04': 1322,
     'exam-rank-05': 1323,
     'exam-rank-06': 1324,
     'cub3d': 1326,
     '42cursus-get_next_line': 1327,
     'ft_server': 1328,
     'ft_services': 1329,
     'libasm': 1330,
     '42cursus-minishell': 1331,
     'webserv': 1332,
     '42cursus-philosophers': 1334,
     'ft_containers': 1335,
     'ft_irc': 1336,
     'ft_transcendence': 1337,
     'cpp-module-00': 1338,
     'cpp-module-01': 1339,
     'cpp-module-02': 1340,
     'cpp-module-03': 1341,
     'cpp-module-04': 1342,
     'cpp-module-05': 1343,
     'cpp-module-06': 1344,
     'cpp-module-07': 1345,
     'cpp-module-08': 1346
}
CURSUS_42_ID = 21

EXCLUDE_USERS = [
    'doosoo',
    'soomhan',
    'smhan',
    'soohan',
    'krsonu',
    'mihwangb',
    'tehan',
    'trandom',
    'ruiz',
    'doby',
    'jai',
    'soomihan',
    'donlee',
    "jalee",
    'ssomin',
    'krisonu',
    'hahantes',
    'hhantest',
    'kedu',
    'teedu',
    'tedu',
    'hwedu',
    'jedu',
    'kyuhong',
    'kyhong',
    'hwlee',
    'bedu',
    'jungbong',
    'insong',
    'hekim',
    'mnyo',
    'jungbong'
]

In [3]:
def get_all_list():
    cadets = []
    idx = 1
    while True:
        d = api.path("cursus/21/users", filter={"primary_campus_id":29}, page={"size":100, "number":idx}).get()
        clear_output(wait = True)
        if len(d) < 1:
            break
        cadets += d
        sleep(0.3)
        idx += 1
    return (cadets)

In [4]:
#get all cadets (cadets + dummies) list
L_all_cadets_list = get_all_list()

https://api.intra.42.fr/v2/cursus/21/users?filter[primary_campus_id]=29&page[size]=100&page[number]=6


In [5]:
def D_get_cadet_data_form(all_cadets_list):
    cadet_data = {}
    for c in all_cadets_list:
        if c['login'] in EXCLUDE_USERS:
            continue
        cadet_data.update({
            c['login']:{
                'user_id' : None,
                ##### progress data####
                '42cursus-libft': [],
                'minirt': [],
                '42cursus-ft_printf': [],
                'netwhat': [],
                'exam-rank-02': [],
                'exam-rank-03': [],
                'exam-rank-04': [],
                'exam-rank-05': [],
                'exam-rank-06': [],
                'cub3d': [],
                '42cursus-get_next_line': [],
                'ft_server': [],
                'ft_services': [],
                'libasm': [],
                '42cursus-minishell': [],
                'webserv': [],
                '42cursus-philosophers': [],
                'ft_containers': [],
                'ft_irc': [],
                'ft_transcendence': [],
                'cpp-module-00': [],
                'cpp-module-01': [],
                'cpp-module-02': [],
                'cpp-module-03': [],
                'cpp-module-04': [],
                'cpp-module-05': [],
                'cpp-module-06': [],
                'cpp-module-07': [],
                'cpp-module-08': [],

                #### k-data #####
                'level': None,
                'blackholed_at': None,
                'remained_days': None,
                'is_blackholed': False,
                'current_exam_rank': None,
                'begin_at': None
            }
        })
    return(cadet_data)

In [6]:
# make a empty dictionary form for each of cadets

cadets_list = D_get_cadet_data_form(L_all_cadets_list)

# GET USERS' PROJECTS PROGRESS DATA

In [9]:
def check_validated_project(pj_user):
    if pj_user['validated?'] == True:
        return True
    return False

def get_project_duration(id: int, end: str):
    tmp = api.path(f"projects_users/{id}").get()
    clear_output(wait = True)
    begin = tmp['teams'][0]['created_at']
    just_date = begin.split('T')[0]
    begin_date = datetime.strptime(just_date, '%Y-%m-%d')
    end_date = datetime.strptime(end.split('T')[0], '%Y-%m-%d')
    return (end_date - begin_date).days, just_date

def get_projects_users(info, data):
    if 'projects_users' in data:
        if len(data) < 1:
            return
        for pj_user in data['projects_users']:
            if pj_user['cursus_ids'][0] == 21:
                is_validated = check_validated_project(pj_user)
                if is_validated == True:
                    duration, begin_at = get_project_duration(pj_user['id'], pj_user['marked_at'])
                    project_name = pj_user['project']['slug']
                    info[project_name].append(begin_at)
                    info[project_name].append(duration)
                    info[project_name].append(pj_user['occurrence'])
                    info[project_name].append(pj_user['final_mark'])
#                     info[f"{pj_user['project']['slug']}"].append(begin_at)
#                     info[f"{pj_user['project']['slug']}"].append(duration)
#                     info[f"{pj_user['project']['slug']}"].append(pj_user['occurrence'])
#                     info[f"{pj_user['project']['slug']}"].append(pj_user['final_mark'])
    else:
        return

# GET USERS' K-DATA

In [7]:
def day_difference(d1, d2):
    start = datetime(int(d1[0:4]), int(d1[5:7]), int(d1[8:10]))
    end = datetime(int(d2[0:4]), int(d2[5:7]), int(d2[8:10]))
    delta = end - start
    return(delta.days)

In [8]:
def get_blackhole_info(cursus_data, info):
    bh = cursus_data['blackholed_at'][0:10]
    today = datetime.today().strftime("%Y-%m-%d")
    if cursus_data['end_at'] != None:
        info['is_blackholed'] = True
    info['blackholed_at'] = bh
    info['remained_days'] = day_difference(today, bh)
    
def get_kdata(info, data):
    info['user_id'] = data['id']
    cursus_users_data = data['cursus_users']
    for index in (range(len(cursus_users_data))):
        if cursus_users_data[index]['cursus_id'] == 21:
            cud = cursus_users_data[index]
            info['level'] = cud['level']
            info['begin_at'] = cud['begin_at'][0:7]
            get_blackhole_info(cud, info)
            break

# Entry Point

In [10]:
error_list = []
for cadet, info in cadets_list.items():
    try:    
        data = api.path(f"users/{cadet}").get()
        clear_output(wait = True)
    except Exception as e:
        # save the list to file
        error_list += [cadet, e]
    get_projects_users(info, data)
    get_kdata(info, data)
    sleep(0.2)

#     with open ("error_list.json", "w") as f:
#         f.write(json.dumps(error_list))

https://api.intra.42.fr/v2/projects_users/1848956?page[size]=100&page[number]=1


# Blackhole인원

In [109]:
def get_by_blackholed(cadets_list):
    bh_list = {
        "non-blackholed" : [],
        "blackholed" : [],
        "blackholed_at" : [],
    }
    for c in cadets_list:
        if cadets_list[c]['is_blackholed'] == True:
            bh_list['blackholed'].append(c)
            bh_list['blackholed_at'].append(cadets_list[c]['blackholed_at'])
        else:
            bh_list['non-blackholed'].append(c)

    return bh_list

# Level 별 인원

In [118]:
def get_by_level(cadets_list):
    by_level = {}
    for c in cadets_list:        
        c_level = int(cadets_list[c]['level'])
        if c_level in by_level.keys():
            by_level[c_level].append(c)
        else:
            by_level[c_level] = []
            by_level[c_level].append(c)
    return(by_level)

# Project 별 완료학생현황

In [ ]:
def get_by_projects(cadets_list):
    project = {}
    for c in cadets_list:
        for p in CURSUS_42_INNER_CIRCLE.keys():
            if not p in project.keys():
                project[p] = {}
            if cadets_list[c][p] != []:
                project[p][c] = cadets_list[c][p]

    return(project)

 `cadet_list` : 가공된 42서울 전체 학생 데이터

## Separate by Piscine Date

In [86]:
def sep_by_piscineDate(cadets_list, piscineDate):
    new_list = {}
    for c in cadets_list:
        if piscineDate in cadets_list[c]['begin_at']:
            new_list[c] = cadets_list[c]
    return new_list

oneone = sep_by_piscineDate(cadets_list, "2020-02")
onetwo = sep_by_piscineDate(cadets_list, "2020-06")
twoone = sep_by_piscineDate(cadets_list, "2020-09")

print("Piscine 별 합격 인원")
print("------------------")
print("1기1차 : " + str(len(oneone)))
print("1기2차 : " + str(len(onetwo)))
print("2기1차 : " + str(len(twoone)))

Piscine 별 합격 인원
------------------
1기1차 : 205
1기2차 : 46
2기1차 : 147


# AGU users

In [185]:
def agu_ongoing_list(agu_list):
    list_agu_login = []
    for s_id in agu_list:
        d = api.path(f"users/{s_id}").get()
        clear_output(wait = True)
        sleep(0.3)
        if d["login"] in EXCLUDE_USERS:
            continue
        expected_date = (agu_list[s_id]['expected_end_date'])[0:10]
        today = datetime.today().strftime("%Y-%m-%d")
        days_til_agu = day_difference(today, expected_date)
        if days_til_agu > 0:
            list_agu_login.append(d['login'])
    return(list_agu_login)

def agu_except_covidcase(raw_data):    
    agu_list = {}
    for i in raw_data:
        s_id = i['user_id']
        if (i['created_at'])[0:10] != "2020-05-08":
            agu_list[s_id] = {
                "created_at" : i['created_at'],
                "updated_at" : i['updated_at'],
                "is_free" : i['is_free'],
                "end_date" : i['end_date'],
                "expected_end_date" : i['expected_end_date']
            }
    return(agu_list)

def get_agu_raw_data():
    idx = 1
    users = []
    while True:
        data = api.path("campus/29/anti_grav_units_users", page={"size":100, "number": idx}).get()
        clear_output(wait = True)
        sleep(0.3)
        if len(data) < 1:
            break
        users += data
        idx += 1
    return(users)

In [186]:
agu_raw_data = get_agu_raw_data()
agu_except_covid_list = agu_except_covidcase(agu_raw_data)
agu_ongoing = agu_ongoing_list(agu_except_covid_list)
print(len(agu_ongoing))
print(agu_ongoing)

2
['jdo', 'ishin']


# Piscine 별 데이터

### `blackhole` 

In [178]:
oneone_blackholed = get_by_blackholed(oneone)
onetwo_blackholed = get_by_blackholed(onetwo)
print("1기1차 블랙홀 인원")
print("------------------")
print("블랙홀 : " + str(len(oneone_blackholed['blackholed'])))
print("생존인 : " + str(len(oneone_blackholed['non-blackholed'])))
print("\n")
print("1기2차 블랙홀 인원")
print("------------------")
print("블랙홀 : " + str(len(onetwo_blackholed['blackholed'])))
print("생존인 : " + str(len(onetwo_blackholed['non-blackholed'])))

1기1차 블랙홀 인원
------------------
블랙홀 : 17
생존인 : 188


1기2차 블랙홀 인원
------------------
블랙홀 : 0
생존인 : 46


In [127]:
df1 = pd.DataFrame(dict([ (k, pd.Series(v)) for k,v in oneone_blackholed.items() ]))
print(df1)

    non-blackholed blackholed blackholed_at
0           jijung    yejchoi    2020-06-10
1            sanam      gwlee    2020-08-08
2          taekkim     hjeong    2020-06-10
3             ykoh    sangkim    2020-06-10
4           japark      kyang    2020-08-09
..             ...        ...           ...
183          hjung        NaN           NaN
184        yonglee        NaN           NaN
185          kukim        NaN           NaN
186       yeonhlee        NaN           NaN
187        minckim        NaN           NaN

[188 rows x 3 columns]


### `level`

In [177]:
oneone_by_level = get_by_level(oneone)
onetwo_by_level = get_by_level(onetwo)
print("1기1차 level 별 인원")
print("------------------")
print("0 <= level < 1 : " + str(len(oneone_by_level[0])))
print("1 <= level < 2 : " + str(len(oneone_by_level[1])))
print("2 <= level < 3 : " + str(len(oneone_by_level[2])))
print("3 <= level < 4 : " + str(len(oneone_by_level[3])))
print("4 <= level < 5 : " + str(len(oneone_by_level[4])))
print("5 <= level < 6 : " + str(len(oneone_by_level[5])))
print("\n")
print("1기2차 level 별 인원")
print("------------------")
print("0 <= level < 1 : " + str(len(onetwo_by_level[0])))
print("1 <= level < 2 : " + str(len(onetwo_by_level[1])))
print("2 <= level < 3 : " + str(len(onetwo_by_level[2])))
print("3 <= level < 4 : " + str(len(onetwo_by_level[3])))


1기1차 level 별 인원
------------------
0 <= level < 1 : 10
1 <= level < 2 : 71
2 <= level < 3 : 70
3 <= level < 4 : 37
4 <= level < 5 : 11
5 <= level < 6 : 6


1기2차 level 별 인원
------------------
0 <= level < 1 : 3
1 <= level < 2 : 32
2 <= level < 3 : 9
3 <= level < 4 : 2


In [128]:
df = pd.DataFrame(dict([ (k, pd.Series(v)) for k,v in oneone_by_level.items() ]))
print(df)
# df.to_csv(f"~/Desktop/by_level.csv")

          2        5        3         1       4        0
0    jijung    sanam  seogkim      seoh    iwoo  yejchoi
1   taekkim  eunhkim  seunkim  joohylee  hyeyoo   hjeong
2      ykoh     jhur  junelee   yongkim   hjeon  sangkim
3    japark  jujeong   taelee     hyejo   hpark     hajo
4    jehong   yopark    dakim     ishin  sebaek  hyuchoi
..      ...      ...      ...       ...     ...      ...
66  sunpark      NaN      NaN       ina     NaN      NaN
67    hjung      NaN      NaN    shpark     NaN      NaN
68  yonglee      NaN      NaN   dachung     NaN      NaN
69    kukim      NaN      NaN      hyim     NaN      NaN
70      NaN      NaN      NaN  yeonhlee     NaN      NaN

[71 rows x 6 columns]


### `subject`

In [187]:
# 하.. 서클별로 정리해서 코드로 프린트해보자..

In [175]:
oneone_all_projects = get_by_projects(oneone)
onetwo_all_projects = get_by_projects(onetwo)

print("1기1차 Project 별 통과인원")
print("-----------------------")
print("libft : " + str(len(oneone_all_projects['42cursus-libft'])))
print("netwhat : " + str(len(oneone_all_projects['netwhat'])))
print("ft_printf : " + str(len(oneone_all_projects['42cursus-ft_printf'])))
print("get_next_line : " + str(len(oneone_all_projects['42cursus-get_next_line'])))
print("-----------------------")
print("ft_server : " + str(len(oneone_all_projects['ft_server'])))
print("cub3d : " + str(len(oneone_all_projects['cub3d'])))
print("minirt : " + str(len(oneone_all_projects['minirt'])))
print("exam-rank-02 : " + str(len(oneone_all_projects['exam-rank-02'])))
print("-----------------------")
print("libasm : " + str(len(oneone_all_projects['libasm'])))
print("minishell : " + str(len(oneone_all_projects['42cursus-minishell'])))
print("ft_services : " + str(len(oneone_all_projects['ft_services'])))
print("exam-rank-03 : " + str(len(oneone_all_projects['exam-rank-03'])))
print("-----------------------")
print("philosophers : " + str(len(oneone_all_projects['42cursus-philosophers'])))
print("cpp-module-00 : " + str(len(oneone_all_projects['cpp-module-00'])))
print("cpp-module-01 : " + str(len(oneone_all_projects['cpp-module-01'])))
print("cpp-module-02 : " + str(len(oneone_all_projects['cpp-module-02'])))
print("cpp-module-03 : " + str(len(oneone_all_projects['cpp-module-03'])))
print("cpp-module-04 : " + str(len(oneone_all_projects['cpp-module-04'])))
print("cpp-module-05 : " + str(len(oneone_all_projects['cpp-module-05'])))
print("cpp-module-06 : " + str(len(oneone_all_projects['cpp-module-06'])))
print("cpp-module-07 : " + str(len(oneone_all_projects['cpp-module-07'])))
print("cpp-module-08 : " + str(len(oneone_all_projects['cpp-module-08'])))
print("exam-rank-04 : " + str(len(oneone_all_projects['exam-rank-04'])))
print("-----------------------")
print("ft_containers : " + str(len(oneone_all_projects['ft_containers'])))
print("ft_irc : " + str(len(oneone_all_projects['ft_irc'])))
print("ft_transcendence : " + str(len(oneone_all_projects['ft_transcendence'])))
print("webserv : " + str(len(oneone_all_projects['webserv'])))
print("exam-rank-05 : " + str(len(oneone_all_projects['exam-rank-05'])))
print("-----------------------")
print("exam-rank-06 : " + str(len(oneone_all_projects['exam-rank-06'])))
print("\n\n")
print("1기2차 Project 별 통과인원")
print("-----------------------")
print("libft : " + str(len(onetwo_all_projects['42cursus-libft'])))
print("netwhat : " + str(len(onetwo_all_projects['netwhat'])))
print("ft_printf : " + str(len(onetwo_all_projects['42cursus-ft_printf'])))
print("get_next_line : " + str(len(onetwo_all_projects['42cursus-get_next_line'])))
print("-----------------------")
print("ft_server : " + str(len(onetwo_all_projects['ft_server'])))
print("cub3d : " + str(len(onetwo_all_projects['cub3d'])))
print("minirt : " + str(len(onetwo_all_projects['minirt'])))
print("exam-rank-02 : " + str(len(onetwo_all_projects['exam-rank-02'])))
print("-----------------------")
print("libasm : " + str(len(onetwo_all_projects['libasm'])))
print("minishell : " + str(len(onetwo_all_projects['42cursus-minishell'])))
print("ft_services : " + str(len(onetwo_all_projects['ft_services'])))
print("exam-rank-03 : " + str(len(onetwo_all_projects['exam-rank-03'])))
print("-----------------------")
print("philosophers : " + str(len(onetwo_all_projects['42cursus-philosophers'])))
print("cpp-module-00 : " + str(len(onetwo_all_projects['cpp-module-00'])))
print("cpp-module-01 : " + str(len(onetwo_all_projects['cpp-module-01'])))
print("cpp-module-02 : " + str(len(onetwo_all_projects['cpp-module-02'])))
print("cpp-module-03 : " + str(len(onetwo_all_projects['cpp-module-03'])))
print("cpp-module-04 : " + str(len(onetwo_all_projects['cpp-module-04'])))
print("cpp-module-05 : " + str(len(onetwo_all_projects['cpp-module-05'])))
print("cpp-module-06 : " + str(len(onetwo_all_projects['cpp-module-06'])))
print("cpp-module-07 : " + str(len(onetwo_all_projects['cpp-module-07'])))
print("cpp-module-08 : " + str(len(onetwo_all_projects['cpp-module-08'])))
print("exam-rank-04 : " + str(len(onetwo_all_projects['exam-rank-04'])))
print("-----------------------")
print("ft_containers : " + str(len(onetwo_all_projects['ft_containers'])))
print("ft_irc : " + str(len(onetwo_all_projects['ft_irc'])))
print("ft_transcendence : " + str(len(onetwo_all_projects['ft_transcendence'])))
print("webserv : " + str(len(onetwo_all_projects['webserv'])))
print("exam-rank-05 : " + str(len(onetwo_all_projects['exam-rank-05'])))
print("-----------------------")
print("exam-rank-06 : " + str(len(onetwo_all_projects['exam-rank-06'])))

1기1차 Project 별 통과인원
-----------------------
libft : 195
netwhat : 162
ft_printf : 133
get_next_line : 183
-----------------------
ft_server : 93
cub3d : 50
minirt : 5
exam-rank-02 : 48
-----------------------
libasm : 37
minishell : 20
ft_services : 19
exam-rank-03 : 16
-----------------------
philosophers : 10
cpp-module-00 : 13
cpp-module-01 : 11
cpp-module-02 : 11
cpp-module-03 : 10
cpp-module-04 : 9
cpp-module-05 : 9
cpp-module-06 : 9
cpp-module-07 : 8
cpp-module-08 : 6
exam-rank-04 : 5
-----------------------
ft_containers : 2
ft_irc : 0
ft_transcendence : 0
webserv : 0
exam-rank-05 : 0
-----------------------
exam-rank-06 : 0



1기2차 Project 별 통과인원
-----------------------
libft : 43
netwhat : 17
ft_printf : 13
get_next_line : 23
-----------------------
ft_server : 4
cub3d : 2
minirt : 0
exam-rank-02 : 4
-----------------------
libasm : 2
minishell : 0
ft_services : 2
exam-rank-03 : 1
-----------------------
philosophers : 0
cpp-module-00 : 0
cpp-module-01 : 0
cpp-module-02 : 0
cp

In [166]:
for p in CURSUS_42_INNER_CIRCLE.keys():
    project_df = pd.DataFrame.from_dict(
        oneone_all_projects[p],
        orient="index",
        columns=['begin_at','duration','occurance', 'final_mark']
    )
    print(project_df)

            begin_at  duration  occurance  final_mark
jijung    2020-02-24        52          1         115
sanam     2020-04-04         5          2         115
taekkim   2020-04-04        10          1         115
ykoh      2020-02-24        73          1         115
japark    2020-04-10        10          3         100
...              ...       ...        ...         ...
hjung     2020-02-25        54          2         115
yonglee   2020-04-03        22          4         115
kukim     2020-02-25        67          1         115
yeonhlee  2020-02-24        79          0         115
minckim   2020-02-24        73          3         115

[195 rows x 4 columns]
          begin_at  duration  occurance  final_mark
dakim   2020-05-18        82          2         106
kylee   2020-08-31         0          0         102
nakim   2020-05-12         6          0         100
gihkim  2020-05-08        35          0         105
seolim  2020-07-08        14          0         100
           begin

### `exam`

In [176]:
print("1기1차 exam 별 통과인원")
print("------------------")
print("exam-rank-02 : " + str(len(oneone_all_projects['exam-rank-02'])))
print("exam-rank-03 : " + str(len(oneone_all_projects['exam-rank-03'])))
print("exam-rank-04 : " + str(len(oneone_all_projects['exam-rank-04'])))
print("exam-rank-05 : " + str(len(oneone_all_projects['exam-rank-05'])))
print("exam-rank-06 : " + str(len(oneone_all_projects['exam-rank-06'])))
print("\n\n")
print("1기2차 exam 별 통과인원")
print("------------------")
print("exam-rank-02 : " + str(len(onetwo_all_projects['exam-rank-02'])))
print("exam-rank-03 : " + str(len(onetwo_all_projects['exam-rank-03'])))
print("exam-rank-04 : " + str(len(onetwo_all_projects['exam-rank-04'])))
print("exam-rank-05 : " + str(len(onetwo_all_projects['exam-rank-05'])))
print("exam-rank-06 : " + str(len(onetwo_all_projects['exam-rank-06'])))

1기1차 exam 별 통과인원
------------------
exam-rank-02 : 48
exam-rank-03 : 16
exam-rank-04 : 5
exam-rank-05 : 0
exam-rank-06 : 0



1기2차 exam 별 통과인원
------------------
exam-rank-02 : 4
exam-rank-03 : 1
exam-rank-04 : 0
exam-rank-05 : 0
exam-rank-06 : 0
